# 3. SQL para preparación de datos

# 3.1 Introducción
Según Forbes, los analistas de datos pasan casi el 80% de su tiempo preparando datos. Trabajar con datos mal organizados puede resultar en análisis incorrectos. Aquí es donde SQL se vuelve una herramienta poderosa, ayudándote a ordenar y limpiar los datos eficientemente.

En este capítulo, aprenderás a juntar datos de diferentes fuentes usando las funciones JOIN y UNION de SQL. También conocerás herramientas que te ayudarán a limpiar los datos, como las funciones CASE WHEN, COALESCE, NULLIF y LEAST/GREATEST. Y para terminar, veremos cómo evitar información duplicada usando el comando DISTINCT.

Todo esto te permitirá trabajar con datos más limpios y precisos.

In [16]:
# @title Preparación del entorno
!pip install tabulate
import sqlite3
from tabulate import tabulate

# Descargamos la base de datos sqlda
!curl https://raw.githubusercontent.com/limspiga/data-modeling/main/db/sqlda.sql -O

# Código auxiliar. Nota: ejecutar cuando se carge este libro.
def sql_exec_query(query):
  conn = sqlite3.connect('sqlda.sql')
  cur = conn.cursor()
  try:
    cur.execute(query)
    headers = [column[0] for column in cur.description]
    print(tabulate(cur,  headers=headers))
    # conn.commit()
  except Exception as e:
    print(str(e))
    # conn.rollback()
  cur.close()
  conn.close()

def sql_exec(query):
  conn = sqlite3.connect('sqlda.sql')
  cur = conn.cursor()
  try:
      cur.execute(query)
      conn.commit()
      print("Query executed successfully")
  except sqlite3.Error as e:
      print(str(e))
      conn.rollback()
  cur.close()
  conn.close()

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 60.5M  100 60.5M    0     0  24.9M      0  0:00:02  0:00:02 --:--:-- 24.9M


# 3.2 Ensamble de Datos

En el segundo capítulo abordaste operaciones básicas en SQL para el manejo de una sola tabla. En esta sección, aprenderás a trabajar con datos distribuidos en múltiples tablas, utilizando uniones y agrupaciones para consolidar la información.

## 3.2.1  Conexión de Tablas con JOIN
En muchas situaciones, la información que queremos no está toda en una sola tabla. Usar un simple comando SELECT en una tabla no es suficiente. Pero no hay de qué preocuparse, porque SQL nos da herramientas para unir tablas relacionadas, como el comando JOIN.

Para que lo veas más claro, imagina que tenemos dos tablas en la base de datos ZoomZoom:

- una de concesionarios y otra de
- vendedores.

In [ ]:
sql_exec_query('''
  PRAGMA table_info(dealerships);
''')

# Figura 3.1: Estructura de la tabla de concesionarios

  cid  name            type        notnull  dflt_value      pk
-----  --------------  --------  ---------  ------------  ----
    0  dealership_id   INTEGER           0                   1
    1  street_address  TEXT              0                   0
    2  city            TEXT              0                   0
    3  state           TEXT              0                   0
    4  postal_code     TEXT              0                   0
    5  latitude        REAL              0                   0
    6  longitude       REAL              0                   0
    7  date_opened     DATETIME          0                   0
    8  date_closed     DATETIME          0                   0


Y la tabla de los vendedores se ve así:

In [ ]:
sql_exec_query('''
  PRAGMA table_info(salespeople);
''')

# Figura 3.2: Estructura de la tabla de vendedores

  cid  name              type        notnull  dflt_value      pk
-----  ----------------  --------  ---------  ------------  ----
    0  salesperson_id    INTEGER           0                   1
    1  dealership_id     INTEGER           0                   0
    2  title             TEXT              0                   0
    3  first_name        TEXT              0                   0
    4  last_name         TEXT              0                   0
    5  suffix            TEXT              0                   0
    6  username          TEXT              0                   0
    7  gender            TEXT              0                   0
    8  hire_date         DATETIME          0                   0
    9  termination_date  DATETIME          0                   0


En la tabla que muestra los detalles de los vendedores, hay una columna etiquetada como `dealership_id`. Esta columna tiene un papel muy importante: nos dice a qué concesionario pertenece cada vendedor, conectándolo directamente con una entrada específica en la tabla de concesionarios a través de una coincidencia de `dealership_id`.

Esta relación es posible gracias a un concepto llamado "**clave extranjera**", que en este caso es el `dealership_id` en la tabla de vendedores. Una "clave extranjera" es simplemente un campo que establece una conexión directa con el campo correspondiente (la "clave principal") en otra tabla, creando así un puente entre las dos tablas.

Gracias a este puente, podemos explorar y analizar los datos de formas muy variadas e interesantes. Imagina que quieres saber quiénes son los vendedores que trabajan en los concesionarios de California; este tipo de preguntas se pueden responder fácilmente. Primero, identificarías qué concesionarios están en California y luego usarías esa información para encontrar a los vendedores correspondientes. A continuación, veremos cómo hacerlo con una consulta SQL:


In [17]:
sql_exec_query('''
SELECT *
FROM dealerships
WHERE state='CA';
''')

# Figura 3.3: Concesionarios en California

  dealership_id  street_address              city         state      postal_code    latitude    longitude  date_opened          date_closed
---------------  --------------------------  -----------  -------  -------------  ----------  -----------  -------------------  -------------
              2  808 South Hobart Boulevard  Los Angeles  CA               90005     34.0578     -118.305  2017-01-26 00:00:00  \N
              5  2210 Bunker Hill Drive      San Mateo    CA               94402     37.5245     -122.344  2017-01-26 00:00:00  \N


Ahora que sabes que los únicos dos concesionarios en California tienen los IDs 2 y 5, respectivamente, puedes entonces consultar la tabla de vendedores, de la siguiente manera:


In [ ]:
sql_exec_query('''
SELECT *
FROM salespeople
WHERE dealership_id in (2, 5)
ORDER BY 1 LIMIT 9;
''')

# Figura 3.4: Vendedores en California

  salesperson_id    dealership_id  title    first_name    last_name    suffix    username       gender    hire_date            termination_date
----------------  ---------------  -------  ------------  -----------  --------  -------------  --------  -------------------  ------------------
              23                2           Beauregard    Peschke                bpeschkem      Male      2021-05-09 00:00:00
              51                5           Lanette       Gerriessen             lgerriessen1e  Female    2021-02-18 00:00:00
              57                5           Spense        Pithcock               spithcock1k    Male      2020-08-11 00:00:00
              61                5           Ludvig        Baynam                 lbaynam1o      Male      2019-04-22 00:00:00
              62                2           Carroll       Pudan                  cpudan1p       Female    2019-01-12 00:00:00
              63                2           Adrianne      Otham                 

Aunque puedes obtener la información que buscas con el método actual, implica hacer dos consultas separadas, lo que puede resultar tedioso. Sería mucho más práctico si pudiéramos fusionar la información de los concesionarios y los vendedores en una sola tabla y luego simplemente filtrar los resultados para mostrar solo los de California. Afortunadamente, SQL nos ofrece una solución eficiente para esto: la cláusula JOIN. Esta cláusula nos permite unir dos o más tablas en una, basándonos en criterios específicos que definimos.

## Tipos de Joins
En este capítulo, aprenderás sobre tres joins fundamentales, que se ilustran en la siguiente figura: joins internos, joins externos y cross joins:

<figure>
<center><img src='https://github.com/limspiga/data-modeling/blob/main/images/7b05c1b2-f698-4eea-8593-cfebb8519a76.png?raw=true' width="300" />
<figcaption>
Figura 3.5: Tipos principales de joins</figcaption></center>
</figure>





### Inner Joins

Un "join interno" es una herramienta que usamos para combinar información de dos tablas diferentes, pero solo cuando cumplen con una condición específica que establecemos, llamada "predicado de join". Esta condición, la mayoría de las veces, es una igualdad, como que un número o nombre en una tabla sea igual a un número o nombre en la otra tabla.

Imagina que es como un filtro que solo deja pasar las combinaciones de filas de ambas tablas que cumplen con esta condición. Si una combinación de filas no cumple con la condición que establecimos, simplemente no la incluimos en los resultados finales.

Para hacer un "join interno", cada fila de la primera tabla se revisa junto con cada fila de la segunda tabla para ver si cumplen con la condición de "join". Solo vemos en los resultados las combinaciones que sí la cumplen.

A continuación, veremos cómo se formula esto técnicamente en SQL:

```sql
SELECT {columns}
FROM {table1}
INNER JOIN {table2}
  ON {table1}.{common_key_1}={table2}.{common_key_2};
```



Aquí, {columns} son las columnas que quieres obtener de la tabla unida, {table1} es la primera tabla, {table2} es la segunda tabla, {common_key_1} es la columna en {table1} a la que quieres unirte, y {common_key_2} es la columna en {table2} a la que quieres unirte.

Ahora, regresa a las dos tablas discutidas anteriormente: concesionarios y vendedores. Como se mencionó anteriormente, sería bueno si pudieras agregar la información de la tabla de concesionarios a la tabla de vendedores sabiendo en qué estado está cada concesionario. Por el momento, supone que todos los ID de los vendedores tienen un valor dealership_id válido.

Puedes unir las dos tablas usando una condición de igualdad en el predicado de unión, como sigue:


In [ ]:
sql_exec_query('''
SELECT *
FROM salespeople
INNER JOIN dealerships
  ON salespeople.dealership_id = dealerships.dealership_id
ORDER BY 1 LIMIT 10;
''')

# Figura 3.6: La tabla de vendedores unida a la tabla de concesionarios

  salesperson_id    dealership_id  title    first_name    last_name    suffix    username    gender    hire_date            termination_date      dealership_id  street_address          city          state      postal_code    latitude    longitude  date_opened          date_closed
----------------  ---------------  -------  ------------  -----------  --------  ----------  --------  -------------------  ------------------  ---------------  ----------------------  ------------  -------  -------------  ----------  -----------  -------------------  -------------
               1               17           Electra       Elleyne                eelleyne0   Female    2020-01-26 00:00:00                                   17  2120 Walnut Street      Philadelphia  PA               19092     39.951      -75.177   2017-01-26 00:00:00  \N
               2                6           Montague      Alcoran                malcoran1   Male      2021-08-27 00:00:00                                    6  731



Como puedes ver en el resultado anterior, la tabla es el resultado de unir la tabla de vendedores con la de concesionarios. Nota que la primera tabla listada en la consulta, vendedores, está en el lado izquierdo del resultado, mientras que la tabla de concesionarios está en el lado derecho. Este orden de izquierda-derecha será muy importante en la próxima sección cuando aprendas sobre los joins externos entre tablas. Durante un join externo, si una tabla está en el lado izquierdo o derecho puede afectar el resultado de la consulta. Para un join interno, sin embargo, el orden de las tablas no es importante para los predicados de unión que usan una operación de igualdad.

Ahora, mira las columnas involucradas; dealership_id en la tabla de vendedores coincide con dealership_id en la tabla de concesionarios. Esto muestra cómo se cumple el predicado de unión. Al ejecutar esta consulta de unión, has creado efectivamente un nuevo "superconjunto de datos" que consta de las dos tablas fusionadas donde las dos columnas dealership_id son iguales.

Ahora puedes ejecutar una consulta SELECT sobre este "superconjunto de datos" de la misma manera que una tabla grande usando las cláusulas y palabras clave del Capítulo 2, Los conceptos básicos de SQL para análisis. Por ejemplo, volviendo al problema de múltiples consultas para determinar cuál de las consultas de ventas funciona en California, ahora puedes abordarlo con una consulta fácil:


In [ ]:
sql_exec_query('''
SELECT *
FROM salespeople
INNER JOIN dealerships
  ON salespeople.dealership_id = dealerships.dealership_id
WHERE dealerships.state = 'CA'
ORDER BY 1 LIMIT 9;
''')

# Figura 3.7: Vendedores en California con una consulta

  salesperson_id    dealership_id  title    first_name    last_name    suffix    username       gender    hire_date            termination_date      dealership_id  street_address              city         state      postal_code    latitude    longitude  date_opened          date_closed
----------------  ---------------  -------  ------------  -----------  --------  -------------  --------  -------------------  ------------------  ---------------  --------------------------  -----------  -------  -------------  ----------  -----------  -------------------  -------------
              23                2           Beauregard    Peschke                bpeschkem      Male      2021-05-09 00:00:00                                    2  808 South Hobart Boulevard  Los Angeles  CA               90005     34.0578     -118.305  2017-01-26 00:00:00  \N
              51                5           Lanette       Gerriessen             lgerriessen1e  Female    2021-02-18 00:00:00                     


Esto te da el siguiente resultado, que muestra las primeras filas del conjunto de resultados completo:


Observarás que el resultado en la Figura 3.6 y la Figura 3.7 es casi idéntico, siendo la excepción que la tabla en la Figura 3.7 tiene los datos de los concesionarios añadidos también. Si quieres recuperar solo la parte de la tabla de vendedores de esto, puedes seleccionar las columnas de vendedores usando la siguiente sintaxis de estrella:


In [ ]:
sql_exec_query('''
SELECT salespeople.*
FROM salespeople
INNER JOIN dealerships
  ON dealerships.dealership_id = salespeople.dealership_id
WHERE dealerships.state = 'CA'
ORDER BY 1 LIMIT 9;
''')

# Figura 3.8: Vendedores en California con SELECT alias de tabla

  salesperson_id    dealership_id  title    first_name    last_name    suffix    username       gender    hire_date            termination_date
----------------  ---------------  -------  ------------  -----------  --------  -------------  --------  -------------------  ------------------
              23                2           Beauregard    Peschke                bpeschkem      Male      2021-05-09 00:00:00
              51                5           Lanette       Gerriessen             lgerriessen1e  Female    2021-02-18 00:00:00
              57                5           Spense        Pithcock               spithcock1k    Male      2020-08-11 00:00:00
              61                5           Ludvig        Baynam                 lbaynam1o      Male      2019-04-22 00:00:00
              62                2           Carroll       Pudan                  cpudan1p       Female    2019-01-12 00:00:00
              63                2           Adrianne      Otham                 


Hay otro atajo que puede ayudar al escribir sentencias con varias cláusulas JOIN. Puedes asignar un alias a los nombres de las tablas para evitar escribir el nombre completo de la tabla cada vez. Simplemente escribe el nombre del alias después de la primera mención de la tabla después de la cláusula JOIN, y podrás ahorrar una cantidad considerable de escritura. Por ejemplo, para la consulta anterior, si quisieras asignar un alias a "salespeople" con "s" y a "dealerships" con "d", podrías escribir la siguiente sentencia:

In [ ]:
sql_exec_query('''
SELECT s.*
FROM salespeople s
INNER JOIN dealerships d
  ON d.dealership_id = s.dealership_id
WHERE d.state = 'CA'
ORDER BY 1
LIMIT 9;
''')

  salesperson_id    dealership_id  title    first_name    last_name    suffix    username       gender    hire_date            termination_date
----------------  ---------------  -------  ------------  -----------  --------  -------------  --------  -------------------  ------------------
              23                2           Beauregard    Peschke                bpeschkem      Male      2021-05-09 00:00:00
              51                5           Lanette       Gerriessen             lgerriessen1e  Female    2021-02-18 00:00:00
              57                5           Spense        Pithcock               spithcock1k    Male      2020-08-11 00:00:00
              61                5           Ludvig        Baynam                 lbaynam1o      Male      2019-04-22 00:00:00
              62                2           Carroll       Pudan                  cpudan1p       Female    2019-01-12 00:00:00
              63                2           Adrianne      Otham                 

Alternatively, you could also put the AS keyword between the table name and alias to make the alias more explicit:

In [ ]:
sql_exec_query('''
SELECT s.*
FROM salespeople AS s
INNER JOIN dealerships AS d
  ON d.dealership_id = s.dealership_id
WHERE d.state = 'CA'
ORDER BY 1
LIMIT 9;
''')

  salesperson_id    dealership_id  title    first_name    last_name    suffix    username       gender    hire_date            termination_date
----------------  ---------------  -------  ------------  -----------  --------  -------------  --------  -------------------  ------------------
              23                2           Beauregard    Peschke                bpeschkem      Male      2021-05-09 00:00:00
              51                5           Lanette       Gerriessen             lgerriessen1e  Female    2021-02-18 00:00:00
              57                5           Spense        Pithcock               spithcock1k    Male      2020-08-11 00:00:00
              61                5           Ludvig        Baynam                 lbaynam1o      Male      2019-04-22 00:00:00
              62                2           Carroll       Pudan                  cpudan1p       Female    2019-01-12 00:00:00
              63                2           Adrianne      Otham                 

Ahora que has cubierto los conceptos básicos de las uniones internas (inner joins), es hora de discutir las uniones externas (outer joins).


### Uniones Externas (Outer Joins)
Como se discutió, las uniones internas solo retornarán filas de las dos tablas cuando se cumpla el predicado de unión para ambas tablas, es decir, cuando ambas tablas tengan filas que puedan satisfacer el predicado de unión. De lo contrario, no se retornarán filas de ninguna de las tablas. Puede ocurrir que, a veces, quieras retornar todas las filas de una de las tablas, incluso si la otra tabla no tiene ninguna fila que cumpla con el predicado de unión. En este caso, dado que no hay ninguna fila que cumpla con el predicado de unión, la segunda tabla no retornará nada más que NULL. La unión externa es un tipo de unión en el que todas las filas de al menos una tabla, si cumplen con la condición WHERE de la consulta, se presentarán después de la operación JOIN.

Las uniones externas pueden clasificarse en tres categorías: uniones externas izquierdas (left outer joins), uniones externas derechas (right outer joins) y uniones externas completas (full outer joins):
- Unión externa izquierda (Left outer join): Las uniones externas izquierdas son aquellas en las que se retornará cada fila de la tabla izquierda (es decir, la tabla mencionada primero en una cláusula de unión). Si no se encuentra una fila de la otra tabla (la tabla derecha), se retorna una fila de NULL de la tabla derecha. Las uniones externas izquierdas se realizan utilizando las palabras clave LEFT OUTER JOIN, seguidas de un predicado de unión. Esto también puede escribirse abreviadamente como LEFT JOIN.

  Para mostrar cómo funcionan las uniones externas izquierdas, examina dos tablas: la tabla de clientes (customers) y la tabla de correos electrónicos (emails). Por el momento, asume que no a todos los clientes se les ha enviado un correo electrónico, y quieres enviar correos a todos los clientes que no han recibido un correo electrónico. Puedes utilizar una unión externa izquierda para hacer que eso ocurra, ya que el lado izquierdo de la unión es la tabla de clientes. Para ayudar a gestionar la salida, limitarás esta a las primeras 1,000 filas. Se utiliza el siguiente fragmento de código:

In [ ]:
sql_exec_query('''
SELECT *
FROM
 customers c
LEFT OUTER JOIN
  emails e ON e.customer_id=c.customer_id
ORDER BY
  c.customer_id
LIMIT
10;
''')

# Figure 3.9: Customers left-joined to emails

  customer_id  title    first_name    last_name    suffix    email                      gender    ip_address     phone         street_address       city         state      postal_code    latitude    longitude  date_added             email_id    customer_id  email_subject                               opened    clicked    bounced    sent_date            opened_date          clicked_date
-------------  -------  ------------  -----------  --------  -------------------------  --------  -------------  ------------  -------------------  -----------  -------  -------------  ----------  -----------  -------------------  ----------  -------------  ------------------------------------------  --------  ---------  ---------  -------------------  -------------------  --------------
            1           Arlena        Riveles                ariveles0@stumbleupon.com  F         98.36.172.246                                                                                                   2019-12-19